In [0]:
# sostituisci con il tuo nome e cognome
NOME_COGNOME = "bill_gates"

## Dataset

In this exam, we'll consider the San Francisco Fire Department Calls dataset.

```
/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv
```

## Part 1

Write a function named `prepare_dataframe()` that

- reads the csv file
- excludes the records with `CallType` that occurs less than `200` times in the entire dataset (it is fine to hardcode the values manually)
- computes a new column `CallDayOfWeek` that computes the day of the week (1 Sunday, 7 Saturday) based on the column `CallDate`
- computes a new column `DeltaPriority` that computes the difference between `FinalPriority` and `OriginalPriority` (eg `DeltaPriority=2` when `FinalPriority=4` and `OriginalPriority=2`)
- keeps only the following columns `CallNumber`, `CallType`, `Priority`, `CallDayOfWeek`, `DeltaPriority`, `Delay`
- remove rows where any record is null (you can use `dropna`)

In [0]:
def prepare_dataframe() -> DataFrame:
    # your code here
    return df

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-35732860650646> in <module>
----> 1 def prepare_dataframe() -> DataFrame:
      2     # your code here
      3     return df

NameError: name 'DataFrame' is not defined

## Part 2

Write a function named `define_pipeline()` that

- returns a `Pipeline` object
- the object is initialized with stages that predict the `Delay` via a linear regression based on the following features: `CallType`, `Priority`, `CallDayOfWeek`, `DeltaPriority`. Two of these features can be considered as categorical, which are the 2?

In [0]:
from pyspark.ml import Pipeline

def define_pipeline() -> Pipeline:
    # your code here
    return pipeline

## Part 3

Write a function named `fit_model(df, pipeline)` that
- takes as input
  - `df`, a `DataFrame` object that corresponds to the one that is returned by `prepare_dataframe()`
  - `pipeline`, a `Pipeline` object that corresonds to the one that is returned by `define_pipeline()`
- splits `df` in _train_ and _test_ dataset (80% and 20% respectively using a seed of `42`)
- fits the `pipeline` on the _train_ dataset and runs prediction on the _test_ dataset
- returns the `DataFrame` of the predictions made of only 2 columns: `DelayLabel`, `DelayPrediction`

In [0]:
def fit_model(df: DataFrame, pipeline: Pipeline) -> DataFrame:
    # your code here
    return predictions_df

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-35732860650651> in <module>
----> 1 def fit_model(df: DataFrame, pipeline: Pipeline) -> DataFrame:
      2     # your code here
      3     return predictions_df

NameError: name 'DataFrame' is not defined

## Part 4
Write a function `evaluate_r2(predictions_df)` that
- takes as input a `DataFrame` object that corresponds to the one that is returned by `fit_model()`, ie made of 2 columns `DelayLabel` and `DelayPrediction`
- computes the R squared metric and returns the R2 value

In [0]:
def evaluate_r2(predictions_df: DataFrame) -> float:
    # your code here
    return r2

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-35732860650653> in <module>
----> 1 def evaluate_r2(predictions_df: DataFrame) -> float:
      2     # your code here
      3     return r2

NameError: name 'DataFrame' is not defined

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
from pyspark.sql.functions import to_date, dayofweek

def prepare_dataframe():
    sf_fire_file = "/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"

    fire_schema = StructType(
        [
            StructField('CallNumber', IntegerType(), True),
            StructField('UnitID', StringType(), True),
            StructField('IncidentNumber', IntegerType(), True),
            StructField('CallType', StringType(), True),                  
            StructField('CallDate', StringType(), True),      
            StructField('WatchDate', StringType(), True),
            StructField('CallFinalDisposition', StringType(), True),
            StructField('AvailableDtTm', StringType(), True),
            StructField('Address', StringType(), True),       
            StructField('City', StringType(), True),       
            StructField('Zipcode', IntegerType(), True),       
            StructField('Battalion', StringType(), True),                 
            StructField('StationArea', StringType(), True),       
            StructField('Box', StringType(), True),       
            StructField('OriginalPriority', IntegerType(), True),       
            StructField('Priority', IntegerType(), True),       
            StructField('FinalPriority', IntegerType(), True),       
            StructField('ALSUnit', BooleanType(), True),       
            StructField('CallTypeGroup', StringType(), True),
            StructField('NumAlarms', IntegerType(), True),
            StructField('UnitType', StringType(), True),
            StructField('UnitSequenceInCallDispatch', IntegerType(), True),
            StructField('FirePreventionDistrict', StringType(), True),
            StructField('SupervisorDistrict', StringType(), True),
            StructField('Neighborhood', StringType(), True),
            StructField('Location', StringType(), True),
            StructField('RowID', StringType(), True),
            StructField('Delay', FloatType(), True)
        ]
    )

    df = spark.read.csv(sf_fire_file, header=True, schema=fire_schema)
    return df\
            .where(~df.CallType.isin(['Administrative', 'Train / Rail Fire', 'Lightning Strike (Investigation)']))\
            .withColumn(
                "CallDayOfWeek",
                dayofweek(to_date(df.CallDate, "MM/dd/yyyy"))
            )\
            .withColumn('DeltaPriority', df.FinalPriority - df.OriginalPriority)\
            .select('CallNumber', 'CallType', 'Priority', 'CallDayOfWeek', 'DeltaPriority', 'Delay')\
            .dropna(how='any')

In [0]:
print("evaluate part 1")

df = prepare_dataframe()
expected_columns = [
    'CallDayOfWeek',
    'CallNumber',
    'CallType',
    'Delay',
    'DeltaPriority',
    'Priority'
]
cols = df.columns
cols.sort()
print("are columns equals?")
print(cols == expected_columns)

rows = 4085557
print("same number of rows?")
assertion = (rows == df.count())
print(assertion)

print("no rows to exclude")
assertion = df.where(df.CallType.isin(['Administrative', 'Train / Rail Fire', 'Lightning Strike (Investigation)'])).count() == 0
print(assertion)

print("check row 20110217")
row = df.where(df.CallNumber == 20110217).collect()[0]
print("assert delay == 1")
print( row['DeltaPriority'] == 1 )
print("assert week day == 6")
print( row['CallDayOfWeek'] == 6)

evaluate part 1
are columns equals?
True
same number of rows?
True
no rows to exclude
True
check row 20110217
assert delay == 1
True
assert week day == 6
True


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression

def define_pipeline() -> Pipeline:
    indexer = StringIndexer(
        inputCols=["CallType", "CallDayOfWeek"],
        outputCols=["CallTypeIndex", "CallDayOfWeekIndex"],
        handleInvalid='skip'
    )
    encoder = OneHotEncoder(
        inputCols=["CallTypeIndex", "CallDayOfWeekIndex"],
        outputCols=["CallTypeOhe", "CallDayOfWeekOhe"]
    )
    assembler = VectorAssembler(
        inputCols=["CallTypeOhe", "Priority", "CallDayOfWeekOhe", "DeltaPriority"],
        outputCol="features"
    )
    regressor = LinearRegression(featuresCol="features", labelCol="Delay")
    return Pipeline(
        stages=[indexer, encoder, assembler, regressor]
    )

In [0]:
print("evaluate part 2")

pipeline = define_pipeline()

stages = pipeline.getStages()
print("4 elements in pipeline?")
print(
    len(stages) == 4
)
print('is stage 0 a StringIndexer?')
print(isinstance(stages[0], StringIndexer))

print('is stage 1 a OneHotEncoder')
print(isinstance(stages[1], OneHotEncoder))

print('is stage 2 a VectorAssembler?')
print(isinstance(stages[2], VectorAssembler))

print('is stage 3 a LinearRegression?')
print(isinstance(stages[3], LinearRegression))

print('fits without errors?')
model = pipeline.fit(df.limit(1000))
print(True)

prediction = model.transform(df.where(df.CallNumber == 20110217)).collect()[0]
print('is DeltaPriority 1?')
print(prediction['DeltaPriority'] == 1 or prediction['DeltaPriority'] == '1')
print('has CallTypeOhe')
print('CallTypeOhe' in prediction)

print('has CallDayOfWeekOhe')
print('CallDayOfWeekOhe' in prediction)

print('has features')
print('features' in prediction)

print('has prediction')
print('prediction' in prediction)

evaluate part 2
4 elements in pipeline?
True
is stage 0 a StringIndexer?
True
is stage 1 a OneHotEncoder
True
is stage 2 a VectorAssembler?
True
is stage 3 a LinearRegression?
True
fits without errors?
True
is DeltaPriority 1?
True
has CallTypeOhe
True
has CallDayOfWeekOhe
True
has features
True
has predictions
False


In [0]:
display(predictions)

CallNumber,CallType,Priority,CallDayOfWeek,DeltaPriority,Delay,CallTypeIndex,CallDayOfWeekIndex,CallTypeOhe,CallDayOfWeekOhe,features,prediction
20110217,Medical Incident,1,6,1,2.9333334,0.0,0.0,"Map(vectorType -> sparse, length -> 10, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 1, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 13, indices -> List(0, 10, 11, 12), values -> List(1.0, 1.0, 1.0, 1.0))",6.5364531837891136


In [0]:
pipeline.getStages()

Out[10]: [StringIndexer_3f0c23a9ec5a,
 OneHotEncoder_7a97049b22e6,
 VectorAssembler_124b97c67555,
 LinearRegression_a3929d8c6911]

In [0]:
from pyspark.sql import DataFrame

def fit_model(df: DataFrame, pipeline: Pipeline) -> DataFrame:
    train, test = df.randomSplit([.8, .2], seed=42)
    model = pipeline.fit(train)
    predictions = model.transform(test)
    return predictions\
            .withColumnRenamed('Delay', 'DelayLabel')\
            .withColumnRenamed('prediction', 'DelayPrediction')\
            .select('DelayLabel', 'DelayPrediction')

In [0]:
predictions = fit_model(df, pipeline)
display(fit_model(df, pipeline))

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

def evaluate_r2(predictions_df: DataFrame) -> float:
    evaluator = RegressionEvaluator(
        predictionCol="DelayPrediction",
        labelCol="DelayLabel",
        metricName="r2"
    )

    return evaluator.evaluate(predictions)

In [0]:
print(
        "R2 ", evaluate_r2(predictions)
)

R2  0.005794662725845279
